## TMA cutting
In the XML file provide by miniMIBI, xy of points will be translated into '<RowNumber0 XAttrib="-86900" YAttrib="180340" /\>', by directly x 10000 to make sure all number are integer.


Two slides region,  
upper one:
(827, 609), (827, 1144), (2200, 609), (2200, 1144)  
or  
(827, 609), (220, 1144)

lower one:
(827, 1447), (827, 1989), (2200, 1447), (2200, 1989)  
or  
(827, 1447), (2200, 1989)

#upper = img[609:1144, 827:2200]
#lower = img[1447:1989, 827:2200]


## Selection between different kinds of sample 
Different kinds of samples involve change of the parameters during the experiments.  
  
large TMA: 20 < w, h < 60, 10 < radius < 30  
small TMA: 10 < w, h < 25, 5 < radius < 15  
lymph nodes:  

In [2]:
import cv2
from PIL import Image
import numpy as np
from random import choice

## Sample position and scale
Image size 2600 x 2600 pixel, the real size of hold on MIBIscope is:  
x: [-51.816 (left), 51.689(right)]  unit: mm,  
y: [51.816 (up), -51.689 (down)]  unit: mm

so the scale relationship is 0.0398 mm/pixel

In [3]:
import sys
import copy
import matplotlib.path as mpltPath
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [4]:
def imageToXML(coodinate):
    # transfer coodinate of carousel image to XML file coodinate
    x, y = coodinate
    xmlx = str(round(1035050*x/2600 - 518160))
    xmly = str(round(518160 - 1035050*y/2600))
    return xmlx, xmly
    
def XMLToImage(coodinate):
    # transfer coodinate of XML file to carousel image coodinate
    xmlx, xmly = coodinate
    x = str(round((xmlx+518160)/1035050*2600))
    y = str(round((518160-xmly)/1035050*2600))
    return x, y

def firstPointCoodinatExtractor(xmlName):
    # extract the point you selected from the carousel image, as xml coodinate and return
    xmlfile = ET.ElementTree(file=xmlName)
    root = xmlfile.getroot()
    coodinate = (int(root[0][-1][0].attrib['XAttrib']), int(root[0][-1][0].attrib['YAttrib']))
    return coodinate

def xmlFileWriter(xmlfileName, PointList, imgCoodinate):
    # based on XML file with the point you selected, add all the point recognized from carousel image, then append to the end
    # of that XML file
    realX, realY = firstPointCoodinatExtractor(xmlfileName)
    imgX, imgY = imgCoodinate # in mm unit
    ErrorX = int(imgX) - int(realX)
    ErrorY = int(imgY) - int(realY)
    
    xmlfile = ET.ElementTree(file=xmlfileName)
    root = xmlfile.getroot()
    for i in range(1, len(PointList)+1): # mind here, need change
        a = copy.deepcopy(root[0][4])
        a.attrib = {'PointName': root[0][4].attrib['PointName'] + '_' + str(i)}
        tempX, tempY = imageToXML(PointList[i-1])
        a[0].attrib['XAttrib'] = str(round(int(tempX) + ErrorX))
        a[0].attrib['YAttrib'] = str(round(int(tempY) + ErrorY))
        
        root[0].append(a)
    xmlfile.write('ALL_'+xmlfileName)
    return

In [5]:
def foundAllPoints(img, sampleType, position):
    """
    param: 
    img, the input image read by imread;
    sampleType, the type of sample, large ('LRG') or small ('SML') TMA.
    position, the image region that want to be selected, ('UP', 'DOWN').
    """
    pointsList = []
    if position == 'UP':
        tempList, coodinate = foundPoints(img, 827, 609, sampleType)
    elif position == 'DOWN':
        tempList, coodinate = foundPoints(img, 827, 1447, sampleType)        
    """
    # for future development, for now it can only observe upper or lower slides region
    elif position == 'ALL':
        tempList, coodinate = foundPoints(img, 827, 609, sampleType)
        pointsList += tempList
        tempList, coodinate = foundPoints(img, 827, 1447, sampleType)
        pointsList += tempList
    """
    pointsList += tempList
    return pointsList, coodinate

def foundALLPoints(img, xModify, yModify, sampleType):    
    imgPos = img[yModify:yModify+540, 827:2200]
    gray = cv2.cvtColor(imgPos, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9),0) 
    (_, thresh) = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    
    images, cnts, hierarchy = cv2.findContours(closed.copy(), cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE)
    print("length of cnts: ", len(cnts))
    
    spotList = []
    for i in range(len(cnts)): 
        #x,y,w,h = cv2.boundingRect(cnts[i])
        (x,y),radius = cv2.minEnclosingCircle(cnts[i])
        center = (int(x)+827, int(y)+609)
        radius = int(radius)
        if sampleType == 'LRG':
            if radius > 15 and radius < 30:
                spotList.append(center)
                cv2.circle(img, center, radius, (0,255,0), 2)
            #if w > 20 and h > 20 and w < 60 and h < 60:
                #spotList.append((x+xModify+w//2, y+yModify+h//2))
                #cv2.rectangle(img,(x+xModify,y+yModify),(x+xModify+w,y+yModify+h),(0,255,0),2)
        elif sampleType == 'SML':
            if radius > 5 and radius < 15:
                spotList.append(center)
                cv2.circle(img, center, radius, (0,255,0), 2)
            #if w > 10 and h > 10 and w < 30 and h < 30:
                #spotList.append((x+xModify+w//2, y+yModify+h//2))
                #cv2.rectangle(img,(x+xModify,y+yModify),(x+xModify+w,y+yModify+h),(0,255,0),2)
    randomX, randomY = choice(spotList)
    pt1 = (randomX+7, randomY+7)
    pt2 = (randomX-7, randomY-7)
    cv2.line(img, pt1, pt2, (255,0,0), 2)
    pt1 = (randomX+7, randomY-7)
    pt2 = (randomX-7, randomY+7)
    cv2.line(img, pt1, pt2, (255,0,0), 2)
                
    a = "contoursImage"+str(len(cnts))+".png"
    cv2.imwrite(a, img)
    imgshow = Image.fromarray(img)
    imgshow.show()
    print("length of spots: ", len(spotList), '\n')
    coodinate = (randomX, randomY)
    return spotList, coodinate

##### step1: read the image

In [6]:
#img = cv2.imread("TMA_test2.jpg")
img = cv2.imread("081318_Ionpath_TMA_Tonsil.jpg")

##### step2: process the image and generate all spots
func foundAllPoints(img, sampleType, position)   
Parameters:  
&nbsp;&nbsp;&nbsp;&nbsp;    Image: carousel image that read in last step  
&nbsp;&nbsp;&nbsp;&nbsp;    TMA_type: choose from 'LRG' and 'SML', 'LRG' for 1.5 mm size TMA, 'SML' for 0.6 mm size TMA  
&nbsp;&nbsp;&nbsp;&nbsp;    Postion: 'UP' indicate to process the upper channel of sample holder, 'DOWN' is the lower one.  

In [21]:
spotslist, coodinate = foundAllPoints(img, 'LRG', 'UP')

length of cnts:  64
please select the center of red-colored cross on miniMIBI, and input the x,y for next function.
length of spots:  41 



##### user step
Check the cross labeled region on popped-up carousel image, use SED to adjust your position, then do all setup for the spots (depth/dwelling time, etc.), save the XML file.

##### step3: process the image and generate all spots
foundAllPoints(xmlfileName, PointList, imgCoodinate)   
Parameters:  
&nbsp;&nbsp;&nbsp;&nbsp; xmlfilename: the name of XML file generated in user step

In [28]:
xmlFileWriter('test.xml', spotslist, imageToXML(coodinate))